In [1]:
# https://blog.naver.com/dalgoon02121/222051184805
# https://youngwonhan-family.tistory.com/66   ---  konlpy, kkma, jpype issue

In [1]:
import scipy as sp
import pandas as pd
import numpy as np
import nltk
import pickle
import re

import konlpy
from konlpy.tag import Okt         ; t = Okt()     # 구 트위터

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.pipeline import Pipeline 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')


In [56]:
with open('./with_name/LDA_merged_final_full.pkl', 'rb') as f:
    news_df = pickle.load(f)

In [57]:
news_df['뉴스제목'] = news_df['뉴스제목'].str.replace('\s+', ' ', regex=True)
news_df['뉴스제목'] = news_df['뉴스제목'].map(lambda x: re.sub('[0-9,.!·,…*°>%@#&$<_?.:\“\/\”\’\'\"\^\\‘\|\`\(\)\[\]\{\}]', '', x))
news_df['뉴스제목'] = news_df['뉴스제목'].map(lambda x: re.sub('\u200c', '', x))
news_df['뉴스제목'] = news_df['뉴스제목'].map(lambda x: re.sub('[-–ㆍ]', ' ', x))

stop_words = ['\u200c','\n ','\n',"'",'…',',','[',']','(',')','"','주','에','·','장','-','적',\
              '도','‘','`','가','’','의','이','★','은','“','대','”','한','B','로',\
              '?','선','A','는','!','"…','상','들','제','…"','일','서','명',"'…",'기',\
              '···','소','등','자','전','률','미','...','세','시','안','폭',"…'",'만','억',\
              '눈','더','량','고','인','성','다','감','을','지','수','것','째',\
              '기','···','중','계','왜','총','내','과','젠','또','연','엔','차','할',\
              '새','사','때','..','임','속','’…','G','나','개','원',\
             '달','→','권','?…','간','배','K','저','와','하','/','조','두','분','형',\
             '황','공','&','보','문','익','X','억원',']"','치','산','를','오','해','S','그','된','준','▶',\
             '건','재','반','라','년','초','분','월','신','p','급','줄','경','구','진','올','발','vs','강',\
             '국','난','판','면','"(','`…','살','아','번','텍','팜','Q','메','점','월',\
             'D','비','됐다','채',"]'",'보니','손','확','종','동','팔','타','~','땐','말','요',\
             "',",'스','…`','단','길','회','호','용','듯','최']

news_df['뉴스제목'] = [each_word for each_word in news_df['뉴스제목'] if each_word not in stop_words]

In [14]:

without_dif = news_df['주가변동'].value_counts()[0] - news_df['주가변동'].value_counts()[1]
pd_without = news_df.drop(news_df[news_df['주가변동'] == 0].sample(n=without_dif).index)
print('Number of Price up: {}'.format(news_df['주가변동'].value_counts()[1]))
print('Number of Price down: {}'.format(news_df['주가변동'].value_counts()[0]))

Number of Price up: 285481
Number of Price down: 325992


In [58]:
news_df

,뉴스제목,가장 비중이 높은 토픽,가장 높은 토픽의 비중,주가변동
0,LG생활건강 숨 로시크숨마 황제의 빛 에디션 선봬,1,0.9640,1
1,IRSK이노베이션 내년 시설투자 올해보다 낮은 수준,0,0.9440,1
2,수출입銀 CJ대한통운 해외채권 발행 공동 보증,0,0.6790,0
3,브랜드 아파트 만원대에 마련할 마지막 기회 사천 KCC스위첸,0,0.9012,1
4,LG전자 뉴욕 한복판에서 영화 상영한 사연,1,0.9518,0
...,...,...,...,...
611468,한국항공우주 인니에 억원 규모 KT B 훈련기 공급 계약,0,0.8536,1
611469,하이트진로 세 박태영 사장 승진,0,0.7068,1
611470,특징주 형제의 난 재점화 가능성에롯데지주 강세,1,0.8101,0
611471,제일기획 비상교육과 청소년 대상 마음 교과서 개발,0,0.7944,1


In [19]:
news_df['okt'] = news_df.apply(lambda row: t.morphs(row['뉴스제목']), axis=1)

In [59]:
train_df = news_df.sample(frac = 0.8)
test_df = news_df.drop(train_df.index)

In [60]:
title_train = train_df['뉴스제목'].copy()
price_train = train_df['주가변동'].copy()
title_test = test_df['뉴스제목'].copy()
price_test = test_df['주가변동'].copy()
# 데이터의 80%는 학습셋, 20%는 테스트셋
# title_train, title_test, price_train, price_test = train_test_split(title_df, price_df, test_size=0.2, random_state=0)


In [63]:
title_train

392262         위메프 원더투어 대한항공 할인 이벤트 진행
265632          하이트진로 영리기업 첫 엔젤투자자로 선정
515134       올해 최고의 게임은 게임대상 온라인 투표 시작
18807     작지만 더 탱글한 오리온젤리 더 탱글 마이구미 출시
223036         SK텔레콤 대구국제공항 로밍부스 단독 오픈
Name: 뉴스제목, dtype: object

In [69]:
tfidf = TfidfVectorizer()
news_title = tfidf.fit_transform(news_df['뉴스제목'])


In [70]:
title_train, title_test, price_train, price_test = train_test_split(news_title, news_df['주가변동'], test_size=0.2, random_state=0)

In [71]:
title_train.shape, title_test.shape

((489178, 317128), (122295, 317128))

In [72]:
logistic = LogisticRegression(C=5e1, solver='lbfgs', multi_class='multinomial', random_state=17, n_jobs=4) 
logistic.fit(title_train, price_train)

LogisticRegression(C=50.0, multi_class='multinomial', n_jobs=4, random_state=17)

In [73]:
y_pred = logistic.predict(title_test)
print(accuracy_score(price_test, y_pred))

0.6143832536080788


In [74]:
with open('./saved_models/LogiReg/basic01.dat', 'wb') as fp:     # 쓰기, 바탕화면에 저장됨
    pickle.dump(logistic, fp)
print('저장완료')     # 학습된 모델 저장 완료


저장완료


In [6]:
# 형태소 분석을 위한 함수
# def tokenizer(text):
#     okt = Okt()
#     token_okt = okt.morphs(text)
#     token_okt = [each_word for each_word in token_okt if each_word not in stop_words]
#     return token_okt

# def data_preprocessing():
#     # 수집한 데이터 읽어오기
#     # news_df = pd.read_excel()
#     # 학습셋, 테스트셋 분리
#     title_list = pd_without['뉴스제목'].tolist()
#     price_list = pd_without['주가변동'].tolist()
#     # 데이터의 80%는 학습셋, 20%는 테스트셋
#     title_train, title_test, price_train, price_test = train_test_split(title_list, price_list, test_size=0.2, random_state=0)
#     return title_train, title_test, price_train, price_test

def learning(x_train, y_train, x_test, y_test):
    # 전처리가 끝난 데이터를 단어 사전으로 만들고
    # 리뷰별로 나오는 단어를 파악해서 수치화 (벡터화)해서 학습
    # tfidf, 로지스틱 회귀 이용
    tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer)
    # 로지스틱
    logistic = LogisticRegression(C=2, penalty='l2', random_state=0, verbose=1)     # C의 숫자가 너무 크면 과적합 (기본 1), penalty로 과적합 방지
    pipe = Pipeline([('vect',tfidf),('clf',logistic)])
    # 학습
    pipe.fit(x_train, y_train)
    # 학습 정확도 측정
    y_pred = pipe.predict(x_test)
    print(accuracy_score(y_test, y_pred))
    # 학습한 모델을 저장
    with open('./saved_models/LogiReg/with_pipe.dat', 'wb') as fp:     # 쓰기, 바탕화면에 저장됨
        pickle.dump(pipe, fp)
    print('저장완료')     # 학습된 모델 저장 완료


In [7]:
# 학습 함수
def model_learning():   # 감성분석 모델 생성
    title_train, title_test, price_train, price_test = data_preprocessing()
    learning(title_train, price_train, title_test, price_test)
    
# 사용 함수
def model_using():   # 감성분석 모델 사용
    using()

model_learning()



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.4s finished


0.9660375840006439
저장완료


In [86]:

# 객체를 복원, 저장된 모델 불러오기
with open('./saved_models/LogiReg/basic01.dat','rb') as fp:     # 읽기
    logistic = pickle.load(fp)
# while True :
text = input('뉴스 타이틀을 입력해주세요 : ')
str = [[text]]

# 예측 정확도
r1 = np.max(logistic.predict_proba(str)*100)     # 확률값을 구해서 *100
# 예측 결과
r2 = logistic.predict(str)[0]     # 긍정('1'), 부정('0')
if r2 == '1':
    print('코스피지수는 상승할 것으로 예상됩니다.')
else: 
    print('코스피지수는 하락할 것으로 예상됩니다.')
print('정확도 : %.3f' % r1)
print('------------------------------------------------')

ValueError: Unable to convert array of bytes/strings into decimal numbers with dtype='numeric'

In [ ]:
model_using()